# Clifford Randomized Benchmarking

This tutorial demonstrates how to generate Clifford randomized benchmarking (RB) circuits using `pygsti`. This is the current standard RB protocol first defined by Magesan et al. in ["Scalable and Robust Benchmarking of Quantum Processes"](http://journals.aps.org/prl/abstract/10.1103/PhysRevLett.106.180504). However, note that there are often good reasons to use other RB protocols, particularly for multi-qubit benchmarking: see the following tutorial on ["Direct RB"](https://arxiv.org/abs/1807.07975).

This tutorial does *not* contain an example of how to analyze Clifford RB data within `pyGSTi`; because many different RB procedures use the same analysis, there is a later tutorial on RB data analysis.

The method demonstrated in this tutorial focuses on (holistic) Clifford RB for an arbitrary number of qubits, so it does not allow the user to specify their own Clifford compilation. Instead, it uses in-built Clifford compilation algorithms that work for any number of qubits.

In [1]:
from __future__ import print_function #python 2 & 3 compatibility

import pygsti
from pygsti.extras import rb


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Specifying the device to be benchmarked

To generate Clifford RB circuits, you first need to specify the device that the circuits will be for. Doing this means that the circuits returned will respect device connectivity, and contain only gates in the "native" gate-set of the device.

We do this using a `ProcessorSpec` object: see the earlier tutorial on how to create these. Here we'll demonstrate creating Clifford RB circuits for device with:
- Five qubits, labelled 'Q0', 'Q1', etc.
- Controlled-Z gates in a ring.
- 1-qubit gates consisting of $\sigma_x$ and $\sigma_y$ rotations by $\pm \pi/2$, and an idle gate.

Below, we generate the `ProcessorSpec` for this device:

In [2]:
nQubits = 5 
qubit_labels = ['Q0','Q1','Q2','Q3','Q4'] 
gate_names = ['Gi', 'Gxpi2', 'Gxmpi2', 'Gypi2', 'Gympi2', 'Gcphase'] 
availability = {'Gcphase':[('Q0','Q1'), ('Q1','Q2'), ('Q2','Q3'), 
                           ('Q3','Q4'),('Q4','Q0')]}
pspec = pygsti.obj.ProcessorSpec(nQubits, gate_names, availability=availability, 
                                 qubit_labels=qubit_labels)

## Generating a Clifford RB experiment

We can generate a set of Clifford RB circuits using the `rb.sample.clifford_rb_experiment()` function.

#### Essential parameters
To sample a Clifford RB experiment, it is necessary to specify:
- The Clifford RB lengths to sample circuits at.
- The number of circuits to sample at each length.

A "Clifford RB length" ($m$) is related to the length of the *uncompiled* Clifford circuit. I.e., the number of random $n$-qubit Clifford gates in the circuit. The literature convention is that $m$ corresponds to the number of independent, uniformly random Clifford gates in the uncompiled circuit. So this is the number of Clifford gates in the uncompiled circuit minus 1 (the inversion Clifford gate is uniquely defined by the preceeding sequence).

Here, we use a different convention: $m$ corresponds to the  number of Clifford gates in the uncompiled circuit minus 2. The reason is that a Clifford RB length can be any integer $\geq 0$, which is the convention that we consistently apply to *all* RB methods within `pyGSTi`. Note that rescaling $m$ by adding a constant is entirely a matter of convention, because RB data is fit to the function $P_m = A + Bp^m$ so mapping $m \to m + c$ will not change the obtained value of $p$ (and $p$ fixes the RB error rate).

Let's fix the Clifford RB lengths to 0, 1, 2, 4, 8 and 16 and take the number of circuits at each length to be $k = 10$.

In [3]:
lengths = [0,1,2,4,8,16]
k = 10

#### Optional parameters

The RB samplers in `pyGSTi` allow the user to benchmark a subset of the qubits, by specifying a __`subsetQs`__ list. This then means that a `ProcessorSpec` can be specified for an entire device even if you only wish to benchmark some subset of it. If this is not specified it is assumed that you want RB circuits for holistically benchmarking the entire device.

Obviously, this set of qubits must be connected (otherwise it is not possible to implement $n$-qubit Clifford gates over these $n$ qubits).

Let's demonstrate generating circuits to benchmark 3 of the qubits:

In [4]:
subsetQs = ['Q0','Q1','Q2']

Another important optional parameter is __`randomizeout`__. This specificies whether the perfect output of the circuits should be the input state (assumed to be $0,0,0...$ herein, although if it is a different computational basis state that can be correct for) or a random computational basis state. The standard Clifford RB procedure is a perfect identity sequence, corresponding to the default of `randomizeout` being False. But there are many good reasons to instead set this to True. Here, we'll stick to the standard procedure, and set it to false

In [5]:
randomizeout = False

#### Generating the circuits

We are now ready to generate the RB experiment:

In [6]:
exp_dict = rb.sample.clifford_rb_experiment(pspec, lengths, k, subsetQs=subsetQs, randomizeout=randomizeout)

- Sampling 10 circuits at CRB length 0 (1 of 6 lengths)
  - Number of circuits sampled = 1,2,3,4,5,6,7,8,9,10,
- Sampling 10 circuits at CRB length 1 (2 of 6 lengths)
  - Number of circuits sampled = 1,2,3,4,5,6,7,8,9,10,
- Sampling 10 circuits at CRB length 2 (3 of 6 lengths)
  - Number of circuits sampled = 1,2,3,4,5,6,7,8,9,10,
- Sampling 10 circuits at CRB length 4 (4 of 6 lengths)
  - Number of circuits sampled = 1,2,3,4,5,6,7,8,9,10,
- Sampling 10 circuits at CRB length 8 (5 of 6 lengths)
  - Number of circuits sampled = 1,2,3,4,5,6,7,8,9,10,
- Sampling 10 circuits at CRB length 16 (6 of 6 lengths)
  - Number of circuits sampled = 1,2,3,4,5,6,7,8,9,10,


And that's it!

## What's in the output?

The returned dictionary contains a full specification for the RB circuits to implement on the device defined by `pspec`. This dictionary contains 4 keys:

In [7]:
print(exp_dict.keys())

dict_keys(['spec', 'qubitordering', 'circuits', 'idealout'])


### The sampled circuits

The 'circuits' key is the main output. This is itself a dictionary where the keys are tuples ($m$,$i$) where $m$ is the RB length and $i = 0, 1, \dots, k$ corresponds to the $i^{\rm th}$ circuit at length $m$. 

In [8]:
print(exp_dict['circuits'].keys())

dict_keys([(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (16, 0), (16, 1), (16, 2), (16, 3), (16, 4), (16, 5), (16, 6), (16, 7), (16, 8), (16, 9)])


Let's look at one of the circuits, which is a `Circuit` object. We'll look at the first circuit sampled at Clifford RB length 0, which consists of two $3$-qubit Clifford gates (one uniformly random, the other it's inverse) compiled into the native gates:

In [9]:
print("The first circuit sampled at Clifford RB length 0:")
print("")
print(exp_dict['circuits'][0,0])
print('')
print("The circuit size is: ", exp_dict['circuits'][0,0].size())
print("The circuit depth is: ", exp_dict['circuits'][0,0].depth())
print("The circuit multi-qubit-gate count is: ", exp_dict['circuits'][0,0].multiQgate_count())

The first circuit sampled at Clifford RB length 0:

Qubit Q0 ---|Gympi2|-|Gympi2|-|Gi |-|  Gi  |-|●Q1|-|Gympi2|-|Gympi2|-|  Gi  |-| ●Q1  |-|Gympi2|-|●Q1|-|  Gi  |-|Gi |-|  Gi  |-| ●Q1  |-|  Gi  |-|Gi |-|  Gi  |-| ●Q1  |-|  Gi  |-|Gi |-|  Gi  |-| ●Q1  |-|Gympi2|-|Gi |-| ●Q1  |-|Gympi2|-|Gympi2|-|  Gi  |-|Gi |-|  Gi  |-|  Gi  |-|  Gi  |-|  Gi  |-|●Q1|-|Gympi2|-|Gympi2|-|Gympi2|-|Gxmpi2|-|Gxmpi2|-|Gympi2|-|  Gi  |-|  Gi  |-|Gi |-|  Gi  |-|  Gi  |-|●Q1|-|  Gi  |-|Gi |-|  Gi  |-| ●Q1  |-|Gympi2|-|Gxmpi2|-|Gympi2|-|●Q1|-|Gympi2|-|Gympi2|-|Gympi2|-|●Q1|-|Gympi2|-|Gympi2|-|  Gi  |-| ●Q1  |-|Gympi2|-|Gympi2|-|Gxmpi2|-|Gxmpi2|---
Qubit Q1 ---|Gympi2|-|Gympi2|-|●Q2|-|Gympi2|-|●Q0|-|Gympi2|-| ●Q2  |-|Gympi2|-| ●Q0  |-|Gympi2|-|●Q0|-|Gympi2|-|●Q2|-|Gympi2|-| ●Q0  |-|Gympi2|-|●Q2|-|Gympi2|-| ●Q0  |-|Gympi2|-|●Q2|-|Gympi2|-| ●Q0  |-|Gympi2|-|●Q2|-| ●Q0  |-|Gympi2|-|Gxmpi2|-|Gympi2|-|●Q2|-|Gympi2|-|Gympi2|-|Gxmpi2|-|Gympi2|-|●Q0|-|Gympi2|-|Gympi2|-|Gxmpi2|-|Gxmpi2|-|  Gi  |-|Gxmpi2|-|Gympi2|-|Gympi2|-

Each of these circuits can be converted to OpenQasm of Quil using the methods shown in the tutorial introducing the `Circuit` object. The set of circuits can also be saved to file, using the `GateString` methods. To do this, we first turn the circuits dictionary into a list (note that the ordering is important unless the RB lengths are also written to file: it is essential to record what Clifford RB length each circuit corresponds to, as this is not strictly possible to extract given a circuit over native gates):

In [10]:
circuitlist = [exp_dict['circuits'][m,i] for m in lengths for i in range(k)]

We can then use the `GateString` export function:

In [11]:
pygsti.io.write_gatestring_list("tutorial_files/CliffordRBCircuits.txt",circuitlist,
                                "Clifford RB circuits")

### The other outputs
The output dictionary contains some other things, which can be very important in some situations.

#### The RB specification

One of these is the specification used to generate the circuits:

In [12]:
exp_dict['spec']

{'citerations': 20,
 'compilerargs': [],
 'descriptor': 'A Clifford RB experiment',
 'randomizeout': False,
 'subsetQs': ['Q0', 'Q1', 'Q2']}

This stores all the information necessary to sample new circuits in the same way (and to know how the circuits were generated), when combined with the `ProcessorSpec` used (which is not stored, as this can be a fairly large object).

However there is one warning here: the `compilerargs` specifies what Clifford compiler to use, and when left as an empty list it will use the default option. This will be set to whatever we consider to be the best general-purpose Clifford compiler in `pyGSTi` (there are multiple algorithms), and so this may change in future version of `pyGSTi`. This is important as the Clifford RB error rate is very strongly dependent on the compilation used (as it is an error rate per Clifford gate). Note that we can avoid compilation-dependence complications by instead implementing ["Direct RB"](https://arxiv.org/abs/1807.07975) - see the next tutorial.

This RB-specification dictionary highlights one other aspects of the Clifford sampling function which may sometimes be important: __`citerations`__ is the number of iterations used in our randomized Clifford compilers. Increasing this will often reduce the size and two-qubit-gate count in each compiled Clifford, at the cost of slowing down the circuit sampling. Sometimes it may be useful to increase this value, particularly if you want to implement Clifford RB on a number of qubits that is at the edge of feasability on your device (due to the native gate error rates) as in this case reducing the "cost" of each Clifford gate will be critical.

#### The error-free circuit outputs

Another component of the output dictionary is the ideal outputs of the circuits, which is a dictionary with the same keys as with 'circuits':

In [13]:
exp_dict['idealout'].keys()

dict_keys([(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (16, 0), (16, 1), (16, 2), (16, 3), (16, 4), (16, 5), (16, 6), (16, 7), (16, 8), (16, 9)])

The values are the error-free outcomes of the circuits. Here, because we have left `randomizeout` as False, this is always the bit-string of 3 zeros:

In [14]:
exp_dict['idealout'][0,0]

(0, 0, 0)

But if `randomizeout` is True, these will be random bit-strings. It will not be possible to analyze the results of the RB experiments without these bit-strings: because "success" corresponds to the circuit output being the particular bit stored here. (These bit-strings can always be re-calculated from the circuits, but doing this with `pyGSTi` requires accessing functions that are currently not demonstrated in any tutorial).

Note that the 'idealout' bitstring assumes the input is ideally $0,0,0,\dots$. So if you use an input state other than $0,0,0,\dots$ it will be necessary to correct for this. E.g., if you start in the computational basis state $1,0,0,\dots$ you would need to add this bit-string modulo 2 to the 'idealout' bit-string.

#### The qubit ordering

The final element in the output dictionary is 'qubitordering':

In [15]:
exp_dict['qubitordering']

('Q0', 'Q1', 'Q2')

This is just a tuple that shows which qubit correpsonds to which bit in the 'idealout' bit-tuples. Note that this can also be extracted from the circuit objects and the RB 'spec', but it's useful information to have easily available.